In [1]:
import pandas as pd
from transformers import AutoTokenizer,pipeline, AutoModelForSeq2SeqLM
import faiss
import numpy as np
import torch
import tensorflow as tf
import joblib

c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
# Cargar el modelo y el tokenizador
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [3]:
# Cargar la base de datos de preguntas y respuestas desde un archivo Excel
excel_file_path = 'Base_conocimiento_pre.xlsx'
df = pd.read_excel(excel_file_path)
preguntas = df['Pregunta'].tolist()
respuestas = df['Respuesta'].tolist()

In [4]:
# Verificar la carga de los datos
print(df.head())

                                            Pregunta  \
0  ¿Dónde se encuentra el departamento de computa...   
1  ¿Dónde se encuentra el psicólogo de la univers...   
2    ¿Cada cuánto tiempo se da las becas economicas?   
3  ¿Cuántos accesos exiten en la matriz de la uni...   
4  ¿Cuáles son los Clubs que dispone la universid...   

                                           Respuesta  
0                                    En el bloque H   
1            En bienestar estudiantil, junto al bar   
2                                     Una vez al año  
3             Dos accesos  para vehiculos y peatones  
4  ASOCIACIÓN DE INGENIEROS MECÁNICOS “ASME”, CHA...  


In [5]:
# Función para generar embeddings
def generar_embeddings(textos, tokenizer, modelo):
    embeddings_list = []
    for texto in textos:
        inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = modelo.encoder(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings_list.append(embeddings)
    return np.array(embeddings_list)

In [6]:
# Generar embeddings para todas las preguntas
embeddings = generar_embeddings(preguntas, tokenizer, model)

In [7]:

# Crear el índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

In [8]:
# Función para buscar la pregunta más similar
def buscar_pregunta_similar(input_question, tokenizer, model, index, preguntas, respuestas, k=1):
    input_embedding = generar_embeddings([input_question], tokenizer, model)
    D, I = index.search(input_embedding.astype('float32'), k)
    return [preguntas[i] for i in I[0]], [respuestas[i] for i in I[0]]

In [9]:
# Función para generar respuesta
def generar_respuesta(modelo, tokenizer, pregunta, respuesta):
    input_text = f"Pregunta: {pregunta}\nRespuesta: {respuesta}\nGenera una respuesta más detallada:"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = modelo.generate(**inputs, max_length=150, num_return_sequences=1, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
# Guardar el modelo, tokenizer, índice FAISS y datos
joblib.dump({
    'model': model,
    'tokenizer': tokenizer,
    'faiss_index': index,
    'preguntas': preguntas,
    'respuestas': respuestas
}, 'chatbot_model.joblib')

print("Modelo guardado exitosamente.")

Modelo guardado exitosamente.


In [11]:

# Ejemplo de cómo cargar y usar el modelo guardado
def cargar_modelo():
    loaded_data = joblib.load('chatbot_model.joblib')
    return loaded_data['model'], loaded_data['tokenizer'], loaded_data['faiss_index'], loaded_data['preguntas'], loaded_data['respuestas']


In [12]:

def responder_pregunta(pregunta):
    model, tokenizer, index, preguntas, respuestas = cargar_modelo()
    preguntas_similares, respuestas_similares = buscar_pregunta_similar(pregunta, tokenizer, model, index, preguntas, respuestas)
    respuesta_detallada = generar_respuesta(model, tokenizer, preguntas_similares[0], respuestas_similares[0])
    translator=Translator()
    respuesta_traducida=translator.translate(respuesta_detallada, dest="es").text
    return respuesta_traducida

In [13]:
# Ejemplo de uso
pregunta_usuario = "¿quien es el director de carrera de software?"
respuesta = responder_pregunta(pregunta_usuario)
print(f"Pregunta: {pregunta_usuario}")
print(f"Respuesta: {respuesta}")

c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))

Pregunta: ¿quien es el director de carrera de software?
Respuesta: Pregunta: ¿Cómo es el director del software? Respuesta: Ing. Mauricio Camapaá Genera
